In [9]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# for cluster analysis
from sklearn.cluster import KMeans

# for PCA - Principal Component Analysis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from scipy.cluster.hierarchy import dendrogram,linkage
%matplotlib inline

#Let's read UseArrests.csv

df = pd.read_csv("UsArrests.csv")

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   City      50 non-null     object 
 1   Murder    50 non-null     float64
 2   Assault   50 non-null     int64  
 3   UrbanPop  50 non-null     int64  
 4   Rape      50 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 2.1+ KB


,City,Murder,Assault,UrbanPop,Rape
0,Alabama,13.2,236,58,21.2
1,Alaska,10.0,263,48,44.5
2,Arizona,8.1,294,80,31.0
3,Arkansas,8.8,190,50,19.5
4,California,9.0,276,91,40.6


In [10]:
# checking city column for unexpected/missing values and any misspellings
city_un = df['City'].unique()
city_un.sort()
city_un

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [11]:
# checking murder column for unexpected values

murder_un = df['Murder'].unique()
murder_un.sort()
murder_un

array([ 0.8,  2.1,  2.2,  2.6,  2.7,  3.2,  3.3,  3.4,  3.8,  4. ,  4.3,
        4.4,  4.9,  5.3,  5.7,  5.9,  6. ,  6.3,  6.6,  6.8,  7.2,  7.3,
        7.4,  7.9,  8.1,  8.5,  8.8,  9. ,  9.7, 10. , 10.4, 11.1, 11.3,
       11.4, 12.1, 12.2, 12.7, 13. , 13.2, 14.4, 15.4, 16.1, 17.4])

# Duplicate Rows

Although there are 50 states ('cities' means 'states' here) and 50 rows it's unlikely that there are any duplicate rows, but worth checking anyway.

In [12]:
duplicates = df[df.duplicated()]
print("Duplicated Rows")
duplicates

Duplicated Rows


,City,Murder,Assault,UrbanPop,Rape
